# Experiments and Evaluation of Shap
- Python Notebook to generate DeepShap explanation maps across all correctly predicted samples to plot the temporal
averages in Figure B4 and B5 (for plot protocol run 'Plot_temporalAverageMaps.py')
- For execution via Colab:
    - 1.) create colab account
    - 2.) sync to colab drive (use colab app) and create shortcut in google drive (right click -> organise -> shortcut)
    - 3.) adapt paths in 'Preliminaries'

In [ ]:
# Install packages.
!pip install scipy==1.10.1
!pip install matplotlib==3.5.3
!pip install keras
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 47.8 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 24.0 MB/s eta 0:00:00


In [ ]:
# Import python packages.
import keras
import numpy as np
import matplotlib.pyplot as plt
import shap
import json
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

# import tensorflow.compat.v1.keras.backend as K
# import tensorflow as tf
# tf.compat.v1.disable_eager_execution()


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


Mounted at /content/drive/


In [ ]:
%%capture
# Install a local package.
!pip install -e /content/drive/MyDrive/Climate_X_Quantus/QuantusClimate/. --user

In [ ]:
%%capture
!pip freeze

In [ ]:
%%capture
# Import local package.
import sys
sys.path.insert(0,'/content/drive/My Drive/Climate_X_Quantus/')
import QuantusClimate as quantus

## Functions

In [ ]:
%%capture
import tensorflow.compat.v1.keras.backend as K
from keras.applications.vgg16 import preprocess_input

def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.layers[layer].input, feed_dict)


In [ ]:
def invert_year_output(ypred, startYear, classChunk, yearsall):
    inverted_years = convert_fuzzyDecade_toYear(ypred, startYear,
                                                    classChunk, yearsall)

    return inverted_years

def convert_fuzzyDecade_toYear(label, startYear, classChunk, yearsall):


    print('SELECT END YEAR - HARD CODED IN FUNCTION')
    years = np.arange(startYear - classChunk * 2, yearsall.max() + classChunk * 2)
    # years = np.arange(startYear - classChunk * 2, 2080 + classChunk * 2)
    chunks = years[::int(classChunk)] + classChunk / 2

    return np.sum(label * chunks, axis=1)

In [ ]:
%%capture
from typing import Dict, Any, Tuple

def explain_all(
               model: keras.Sequential,
               XXt: np.ndarray,
               Yyt: np.ndarray,
               method: Any,
               **params
) -> np.ndarray:
    """
    Generate explanation for all inputs
    """

    startYear = params.get('startYear', 1920)
    annType = params['XAI'].get('annType', 'class')

    ### Define prediction error
    yearsUnique = np.unique(Yyt)
    percCutoff = 90
    withinYearInc = 2.
    errTolerance = withinYearInc

    err = Yyt[:, 0] - invert_year_output(model.predict(XXt),
                                             startYear, params['classChunk'], params['yall'])

    base = method[1]["base"]
    if 'MLP'in method[1]["net"]:
      base = base.reshape((len(base),method[1]["lat"]*method[1]["lon"]))
      inputs = XXt.reshape((len(XXt),method[1]["lat"]*method[1]["lon"]))
    else:
      base = base.reshape((len(base),method[1]["lat"],method[1]["lon"],1))
      inputs = XXt.reshape((len(XXt),method[1]["lat"],method[1]["lon"],1))

    exp = shap.DeepExplainer(model, base)

    maps = np.empty(np.shape(XXt))

    for i in np.arange(0, np.shape(XXt)[0]):

      analyzer_output = exp.shap_values(inputs[i,np.newaxis,...],  ranked_outputs=1, check_additivity=False)
      maps[i] = np.array(analyzer_output[0])

    ### Compute the frequency of data at each point and the average relevance
    ### normalized by the sum over the area and the frequency above the 90th
    ### percentile of the map
    yearsUnique = np.unique(Yyt)
    if params['net'] == 'CNN':
        dTM = maps.reshape((yearsUnique.shape[0],int(np.shape(maps)[0]/yearsUnique.shape[0]),np.shape(maps)[1]*np.shape(maps)[2]))
        deepTaylorMaps = maps.reshape((np.shape(maps)[0],np.shape(maps)[1]*np.shape(maps)[2]))
    else:
        dTM = maps.reshape((yearsUnique.shape[0], int(np.shape(maps)[0] / yearsUnique.shape[0]), np.shape(maps)[1]))
        deepTaylorMaps = maps

    summaryX = np.nanmean(dTM, axis = 1)



    return summaryX, dTM, maps


## Preliminaries
- Set raw_path to raw data path
- Set save_path to DeepShap result path
- Set net = 'MLP' for MLP-based evaluation of DeepShap
- Set net = 'CNN' for CNN-based evaluation of DeepShap

In [ ]:
# Set experiment settings.
import yaml

exp_path = '/content/drive/MyDrive/Climate_X_Quantus/Experiment/'
raw_path = '/content/drive/MyDrive/Climate_X_Quantus/Data/'
save_path = '/content/drive/MyDrive/Climate_X_Quantus/Data/Training/'




config = yaml.load(open(exp_path + '/plot_config.yaml'), Loader=yaml.FullLoader)


# Experiment variables.
net = 'CNN'
params = config['params']
params['net'] = net




### Load Data

In [ ]:
# Load the full data object.
all = np.load(raw_path + 'Quantus/%s' + '/0/' + 'Postprocessed_data_ALL.npz', allow_pickle=True)
background= all["Input"].reshape(all["Input"].shape[0], 1, len(all["wh"][0]), len(all["wh"][1]))

# select a set of background examples to take an expectation over.
background = background[np.random.choice(background.shape[0], 100, replace=False)]

# Longitude and latitudes.
lat = all['wh'][0]
lon = all['wh'][1]

del all

In [ ]:
# # Generate explanations.

data = np.load(raw_path + 'Training/' 'Preprocessed_data_%s_CESM1_obs_20CRv3.npz' % net, allow_pickle=True)
ins = data['XtrainS']
inst = data['XtestS']
Ytrain = data['Ytrain']
Ytest = data['Ytest']

# Reshape.
x_batch = np.append(ins, inst, axis=0)
y_batch = np.append(Ytrain, Ytest, axis=0)

### Load model

In [ ]:
from keras.models import load_model
import keras

model = load_model(raw_path + '/Network/' + 'lens_%s_0_T2M_1.tf' % net, compile=False)

# Run the model on a test sample, requiring a compilation.
model.compile(optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
              loss='binary_crossentropy',
              metrics=[keras.metrics.categorical_accuracy],)

The `lr` argument is deprecated, use `learning_rate` instead.


### Create explanations SHAP

In [ ]:
# Reshape Data
if 'MLP' in net:
  backg = background.reshape((len(background),len(lat)*len(lon)))
  x_b= x_batch.reshape((len(x_batch), len(lat)*len(lon)))

else:
  backg = background.reshape((len(background),len(lat),len(lon),1))
  x_b = x_batch.reshape((len(x_batch), len(lat),len(lon),1))

In [ ]:
import shap

params['yall'] = np.arange(1920, 2080 + 1, 1)

# Explanation variables.
xai_methods =[("DeepSHAP", {"base": backg, "lat":len(lat),"lon":len(lon), "net":net}, "DeepSHAP")]

summary, dTM, maps = explain_all(model, x_batch, y_batch, xai_methods[0], **params)

202/202 [==============================] - 7s 2ms/step
SELECT END YEAR - HARD CODED IN FUNCTION


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.



### Save files

In [ ]:
# format.
xaimapsall_m = summary.reshape(1, 1,len(params['yall']), len(lat), len(lon))
xaimapsAll_m = maps.reshape(1, 1, 40, len(params['yall']), len(lat), len(lon))

# Save mean maps.
np.savez(save_path + '%s/DeepShap_UAI_YearlyMaps_1_20ens_T2M_training_ALL_annual.npz' %net, values = xaimapsAll_m)

# Save maps.
np.savez(save_path + '%s/DeepShap_UAI_YearlyMaps_1_20ens_T2M_training_cleaned_annual.npz' %net, values= xaimapsall_m)

